# Visualizing 2D Emotional-Manifolds from Acoustic Features   
## Efthymios Tzinis

In [ ]:
# Load the appropriate modules 
import os, sys, glob
import numpy as np
sys.path.append('../')
import config
sys.path.append(config.BASE_PATH)
from dataloader import fused_features_IEMOCAP as IEMOCAP_loader

sys.path.append(config.PATTERN_SEARCH_MDS_PATH)

In [ ]:
# Loading functions for the whole dataset with emotional utterances 
def get_dataset_in_one_array(features_dic):
    x_all_list = []
    Y_all = []
    for te_speaker, te_data in features_dic.items():  
        x_all_list.append(te_data['x'])
        Y_all += te_data['y']        
        X_all = np.concatenate(x_all_list, axis=0)
    return X_all, Y_all

## Initialize all available Dimensionalality Reduction Methods

In [ ]:
import multidimensional
import multidimensional.common
import multidimensional.mds 
import multidimensional.smacof
from sklearn import manifold, decomposition
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import pprint 
import pandas as pd 

class IdentityData(object):
    def __init__(self):
        pass 
    
    def fit_transform(self, x):
        return x

def get_manifold_methods(target_dim):
    method_n_comp = 66
    radius_barrier = 1e-3
    explore_dim_percent = .9
    starting_radius = 1
    max_turns = 10000
    point_filter = (multidimensional.point_filters.FixedStochasticFilter(keep_percent=1, recalculate_each=10))
    radius_update = (multidimensional.radius_updates.AdaRadiusHalving(tolerance=.5*1e-3, burnout_tolerance=100000))

    mds_obj = multidimensional.mds.MDS(target_dim, point_filter, radius_update, starting_radius=starting_radius, 
                                       radius_barrier=radius_barrier,
                max_turns=max_turns, keep_history=False,
                explore_dim_percent=explore_dim_percent)

    manifold_methods = {
        'Pattern Search MDS': { 'results': {}, 'object': multidimensional.mds.MDS(target_dim, point_filter, 
                                                         radius_update, starting_radius=starting_radius, 
                                                         radius_barrier=radius_barrier, max_turns=max_turns, 
                                                         keep_history=False,
                                                         dissimilarities='precomputed',
                                                         explore_dim_percent=explore_dim_percent)},
        'MDS SMACOF': { 'results': {}, 'object': multidimensional.smacof.MDS(n_components=target_dim, n_init=1, 
                                                 max_iter=max_turns, dissimilarity='euclidean', n_jobs=8)},
        'LTSA': { 'results': {}, 'object': manifold.LocallyLinearEmbedding(method_n_comp, target_dim, 
                                           eigen_solver='auto', method='ltsa',n_jobs=8)},
        'Modified LLE': { 'results': {}, 'object': manifold.LocallyLinearEmbedding(method_n_comp, target_dim, 
                                           eigen_solver='auto', method='modified',n_jobs=8)},
        'Hessian LLE': { 'results': {}, 'object': manifold.LocallyLinearEmbedding(method_n_comp, target_dim, 
                                           eigen_solver='auto', method='hessian',n_jobs=8)},
        'LLE': { 'results': {}, 'object': manifold.LocallyLinearEmbedding(method_n_comp, target_dim, 
                                           eigen_solver='auto', method='standard',n_jobs=8)},
        'Truncated SVD': { 'results': {}, 'object': decomposition.TruncatedSVD(n_components=target_dim)},
        'Spectral Embedding': { 'results': {}, 'object': manifold.SpectralEmbedding(n_components=target_dim, 
                                                                                    n_jobs=8)},
        'TSNE': { 'results': {}, 'object': manifold.TSNE(n_components=target_dim)},
        'ISOMAP': { 'results': {}, 'object': manifold.Isomap(12, target_dim)},
        'Original Data': { 'results': {}, 'object': IdentityData()}

    }
    return manifold_methods

In [ ]:
# Function for creating manifolds visualizations on grid
def create_2d_manifolds_for_all_methods(data_dic, methods_to_test, target_dim=2):
    X_all, Y_all = get_dataset_in_one_array(data_dic)
    X_high = StandardScaler().fit_transform(X_all)
    manifold_methods = get_manifold_methods(target_dim)
    
    results_for_methods = {}
    print X_high.shape
    
    for selected_method in methods_to_test:
        print 'Checking Method: {}'.format(selected_method)
        obj = manifold_methods[selected_method]['object']
        
        try:
            if selected_method == 'Pattern Search MDS':
                d_goal = multidimensional.common.DISTANCE_MATRIX(X_high.astype(np.float64))
                X_low = obj.fit_transform(d_goal)
            else:
                X_low = obj.fit_transform(X_high)
        except Exception as e:
            print "Warning Method: {} did not produce results".format(selected_method)
            print e
            X_low = None
            
        results_for_methods[selected_method] = X_low
        
    return results_for_methods, Y_all

# Compute 2D Manifolds Learned from different Dimensionality Reduction Methods using all combinations of features for EmoDB

In [ ]:
# Load the dataset for different feature sets and create 2D manifolds 
# Find all appropriate files 
data_path = '/home/thymios/all_BERLIN_features/'
berlin_l_feats_p = data_path + 'linear/BERLIN_linear_emobase2010'
berlin_nl_feats_p = os.path.join(data_path, 
             'rqa/utterance/BERLIN-rqa-ad_hoc-tau-7-manhattan-recurrence_rate-0.15-dur-0.02-fs-16000.dat')

RQA_dic = IEMOCAP_loader.get_fused_features([berlin_nl_feats_p])
Linear_dic = IEMOCAP_loader.get_fused_features([berlin_l_feats_p])
Fused_dic = IEMOCAP_loader.get_fused_features([berlin_l_feats_p, berlin_nl_feats_p])

methods_to_test = ['Pattern Search MDS', 'MDS SMACOF','Truncated SVD', 'Spectral Embedding', 'LLE', 
                   'Hessian LLE', 'Modified LLE', 'LTSA', 'ISOMAP', 'TSNE']   
X_lows_for_RQA, Y_RQA = create_2d_manifolds_for_all_methods(RQA_dic, methods_to_test)
X_lows_for_Emobase, Y_Emobase = create_2d_manifolds_for_all_methods(Linear_dic, methods_to_test)
X_lows_for_Fused, Y_fused = create_2d_manifolds_for_all_methods(Fused_dic, methods_to_test)

In [ ]:
# Functions for creating the scatter plots based on the existing Y_values 
def get_xy_pairs_based_on_class(Xm, Ym):
    emotions = {}
    for y in Ym:
        if not y in emotions:
            emotions[y] = {'xs':[], 'ys':[]}
    
    if Xm is None:
        return emotions
    
    for i in np.arange(Xm.shape[0]):
        this_emotion = Ym[i]
        emotions[this_emotion]['xs'].append(Xm[i,0])
        emotions[this_emotion]['ys'].append(Xm[i,1])
    return emotions

def get_xy_for_all(emotions, Ym):
    results_to_plot = {}
    for method, Xm in emotions.items():
        xypairs = get_xy_pairs_based_on_class(Xm, Ym)
        results_to_plot[method] = xypairs
    return results_to_plot

In [ ]:
# Plotly Functions 
import plotly
import plotly.tools as tls
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
plotly.offline.init_notebook_mode()

def create_one_scatter(title, emotions_xy_pairs):
    traces = []
    for emotion, xypairs in emotions_xy_pairs.items():
        trace = go.Scatter(
            x = xypairs['xs'],
            y = xypairs['ys'],
            name = emotion,
            opacity = 0.7,
            mode = 'markers',
            marker = dict(symbol = 'star', size = 10)    
        )
        traces.append(trace)
        
    layout = dict(title = title)
    fig = dict(data=traces, layout=layout)
    plotly.offline.iplot(fig, filename=title)

## Compare 2D Manifolds Leanred from different Dimensionality Reduction Methods when using only RQA features

In [ ]:
results_to_plot = get_xy_for_all(X_lows_for_RQA, Y_RQA)
for method, emotions_xy_pairs in results_to_plot.items():
    create_one_scatter(method, emotions_xy_pairs)

## Compare 2D Manifolds Leanred from different Dimensionality Reduction Methods when using only Emobase features

In [ ]:
results_to_plot = get_xy_for_all(X_lows_for_Emobase, Y_Emobase)
for method, emotions_xy_pairs in results_to_plot.items():
    create_one_scatter(method, emotions_xy_pairs)

## Compare 2D Manifolds Learned from different Dimensionality Reduction Methods when using the Fused feature set

In [ ]:
results_to_plot = get_xy_for_all(X_lows_for_Fused, Y_fused)
for method, emotions_xy_pairs in results_to_plot.items():
    create_one_scatter(method, emotions_xy_pairs)

# Experiments with different speakers from IEMOCAP

In [ ]:
def fuse_excited_happiness(l):
    return ['happy + excited' 
            if (e == 'excited' or e == 'happy') 
            else e for e in l ]

def create_2d_manifolds_for_all_methods_for_speakers(data_dic, methods_to_test, 
                                                     target_dim=2, selected_speakers=['Ses01M', 'Ses02M']):
    X1 = data_dic[selected_speakers[0]]['x']
    Y1 = data_dic[selected_speakers[0]]['y']
    X2 = data_dic[selected_speakers[1]]['x']
    Y2 = data_dic[selected_speakers[1]]['y']
    
    X_all = np.concatenate([X1, X2], axis=0)
    Y_all = Y1 + Y2
    
    X_high = StandardScaler().fit_transform(X_all)
    manifold_methods = get_manifold_methods(target_dim)
    
    results_for_methods = {}
    print X_high.shape
    
    for selected_method in methods_to_test:
        print 'Checking Method: {}'.format(selected_method)
        obj = manifold_methods[selected_method]['object']
        results_for_methods[selected_method] = {'X1':None, 'X2':None}
        
        try:
            if selected_method == 'Pattern Search MDS':
                d_goal = multidimensional.common.DISTANCE_MATRIX(X_high.astype(np.float64))
                X_low = obj.fit_transform(d_goal)
            else:
                X_low = obj.fit_transform(X_high)
        except Exception as e:
            print "Warning Method: {} did not produce results".format(selected_method)
            print e
            
        X1_low = X_low[:X1.shape[0],:]
        X2_low = X_low[X1.shape[0]:,:]
        results_for_methods[selected_method]['X1'] = X1_low
        results_for_methods[selected_method]['X2'] = X2_low
        
    return results_for_methods, Y1, Y2

In [ ]:
# Load IEMOCAP
IEMOCAP_data_path = '/home/thymios/all_TRUE_IEMOCAP_feats/'
l_feats_p = IEMOCAP_data_path + 'linear/IEMOCAP_linear_emobase2010'
nl_feats_p = os.path.join(IEMOCAP_data_path, 
             'utterance/IEMOCAP-rqa-ad_hoc-tau-7-supremum-recurrence_rate-0.15-dur-0.03-fs-16000.dat')
selected_speakers = ['Ses01M', 'Ses02M']
Fused_IEMO_dic = IEMOCAP_loader.get_fused_features([l_feats_p, nl_feats_p])
results_for_methods, Y1, Y2 = create_2d_manifolds_for_all_methods_for_speakers(Fused_IEMO_dic, methods_to_test, 
                                                     target_dim=2, selected_speakers=selected_speakers)

## Visualize two speakers at the same time 


In [ ]:
def create_one_scatter_for_two_speakers(title, speaker_names, emotions_xy_pairs1, emotions_xy_pairs2):
    colors = ['blue', 'red', 'green', 'orange']
    traces = []
    counter = 0
    sorted_emotions = sorted(emotions_xy_pairs1.keys())
    for emotion in sorted_emotions:
        xypairs = emotions_xy_pairs1[emotion]
        trace = go.Scatter(
            x = xypairs['xs'],
            y = xypairs['ys'],
            name = speaker_names[0]+' '+emotion,
            mode = 'markers',
            opacity = 0.5,
            marker = dict(symbol = 'star', size = 10, color=colors[counter])    
        )
        traces.append(trace)
        counter += 1
        
    counter = 0
    for emotion in sorted_emotions:
        xypairs = emotions_xy_pairs2[emotion]
        trace = go.Scatter(
            x = xypairs['xs'],
            y = xypairs['ys'],
            name = speaker_names[1]+' '+emotion,
            mode = 'markers',
            opacity = 0.5,
            marker = dict(size = 10, color=colors[counter])    
        )
        traces.append(trace)
        counter += 1
        
    layout = dict(title = title)
    fig = dict(data=traces, layout=layout)
    plotly.offline.iplot(fig, filename=title)


In [ ]:
Y1, Y2 = fuse_excited_happiness(Y1), fuse_excited_happiness(Y2)
for method, X_dic in results_for_methods.items():
    emotions_xy_pairs1 = get_xy_pairs_based_on_class(X_dic['X1'], Y1)
    emotions_xy_pairs2 = get_xy_pairs_based_on_class(X_dic['X2'], Y2)
    create_one_scatter_for_two_speakers(method, selected_speakers, emotions_xy_pairs1, emotions_xy_pairs2)

### With Speakers of the same Session (Basically the same Dialogue Patterns)

In [ ]:
selected_speakers_same = ['Ses02M', 'Ses02F']
Fused_dic_same = IEMOCAP_loader.get_fused_features([l_feats_p, nl_feats_p])
results_for_methods_same, Y1_same, Y2_same = create_2d_manifolds_for_all_methods_for_speakers(
    Fused_dic_same, methods_to_test, target_dim=2, selected_speakers=selected_speakers_same)

In [ ]:
Y1_same, Y2_same = fuse_excited_happiness(Y1_same), fuse_excited_happiness(Y2_same)
for method, X_dic in results_for_methods_same.items():
    emotions_xy_pairs1 = get_xy_pairs_based_on_class(X_dic['X1'], Y1)
    emotions_xy_pairs2 = get_xy_pairs_based_on_class(X_dic['X2'], Y2)
    create_one_scatter_for_two_speakers(method, selected_speakers_same, emotions_xy_pairs1, emotions_xy_pairs2)

In [ ]:
# MAnifolds in 3d 
results_for_methods_3d, Y1_3d, Y2_3d = create_2d_manifolds_for_all_methods_for_speakers(Fused_IEMO_dic, methods_to_test, 
                                                     target_dim=3, selected_speakers=selected_speakers)

In [ ]:
def get_xy_pairs_based_on_class_3d(Xm, Ym):
    emotions = {}
    for y in Ym:
        if not y in emotions:
            emotions[y] = {'xs':[], 'ys':[], 'zs':[]}
    
    if Xm is None:
        return emotions
    
    for i in np.arange(Xm.shape[0]):
        this_emotion = Ym[i]
        emotions[this_emotion]['xs'].append(Xm[i,0])
        emotions[this_emotion]['ys'].append(Xm[i,1])
        emotions[this_emotion]['zs'].append(Xm[i,2])
    return emotions

def create_one_scatter_for_two_speakers_3d(title, speaker_names, emotions_xy_pairs1, emotions_xy_pairs2):
    colors = ['blue', 'red', 'green', 'orange']
    traces = []
    counter = 0
    sorted_emotions = sorted(emotions_xy_pairs1.keys())
    for emotion in sorted_emotions:
        xypairs = emotions_xy_pairs1[emotion]
        trace = go.Scatter3d(
            x = xypairs['xs'],
            y = xypairs['ys'],
            z = xypairs['zs'],
            name = speaker_names[0]+' '+emotion,
            mode = 'markers',
            opacity = 0.5,
            marker = dict(symbol = 'x', size = 5, color=colors[counter])    
        )
        traces.append(trace)
        counter += 1
        
    counter = 0
    for emotion in sorted_emotions:
        xypairs = emotions_xy_pairs2[emotion]
        trace = go.Scatter3d(
            x = xypairs['xs'],
            y = xypairs['ys'],
            z = xypairs['zs'],
            name = speaker_names[1]+' '+emotion,
            mode = 'markers',
            opacity = 0.5,
            marker = dict(size = 5, color=colors[counter])    
        )
        traces.append(trace)
        counter += 1
        
    layout = dict(title = title)
    fig = dict(data=traces, layout=layout)
    plotly.offline.iplot(fig, filename=title)


In [ ]:
Y1_3d, Y2_3d = fuse_excited_happiness(Y1_3d), fuse_excited_happiness(Y2_3d)
for method, X_dic in results_for_methods_3d.items():
    emotions_xy_pairs1 = get_xy_pairs_based_on_class_3d(X_dic['X1'], Y1_3d)
    emotions_xy_pairs2 = get_xy_pairs_based_on_class_3d(X_dic['X2'], Y2_3d)
    create_one_scatter_for_two_speakers_3d(method, selected_speakers, emotions_xy_pairs1, emotions_xy_pairs2)

# Create the emotional maps from EmoDB in 2d space

In [ ]:
methods_to_show = ['Pattern Search MDS', 'MDS SMACOF', 'Spectral Embedding', 'LLE', 'ISOMAP', 'Truncated SVD']
X_RQA3, Y_RQA3 = create_2d_manifolds_for_all_methods(RQA_dic, methods_to_show)
X_Emobase3, Y_Emobase3 = create_2d_manifolds_for_all_methods(Linear_dic, methods_to_show)
X_Fused3, Y_fused3 = create_2d_manifolds_for_all_methods(Fused_dic, methods_to_show)

In [ ]:
from plotly import tools
def get_one_scatter_2d_traces(title, emotion_data, showlegend=False):
    colors = ['blue', 'red', 'green', 'orange', 'grey', 'magenta', 'black']
    traces = []
    sorted_emotions = sorted(emotion_data.keys())
    counter = 0
    for emotion in sorted_emotions:
        xypairs = emotion_data[emotion]
        trace = go.Scatter(
            x = xypairs['xs'],
            y = xypairs['ys'],
#             z = xypairs['zs'],
            name = emotion,
            legendgroup = emotion,
            mode = 'markers',
            opacity = 0.5,
            marker = dict(symbol = 'diamond', size = 5, color = colors[counter]),
            showlegend = showlegend
        )
        counter += 1
        traces.append(trace)
    return traces

def create_subplots3ds_emodb(Xs, Y, methods_to_show):
    
    if len(methods_to_show) == 6:
        pass 
    else: 
        raise IndexError('Not 6 plots to produce')
        
    fig = tools.make_subplots(rows=3, cols=2,
                             subplot_titles=[x if not x == 'Spectral Embedding' else 'Spectral Clustering'
                                             for x in methods_to_show]
                                             ,
                             horizontal_spacing = 0.01, vertical_spacing = 0.1)    

    legend_shower = methods_to_show[-1]
    
    row = 1
    place_dic = {}
    for k, method in enumerate(methods_to_show):
        if k % 2: 
            place_dic[method] = (row, 2)
            row += 1
        else:
            place_dic[method] = (row, 1)

    for method in methods_to_show:
        Xm = Xs[method]
        emotion_data = get_xy_pairs_based_on_class(Xm, Y)
        if legend_shower == method:
            print legend_shower
            this_traces = get_one_scatter_2d_traces(method, emotion_data, showlegend=True)
        else:
            this_traces = get_one_scatter_2d_traces(method, emotion_data, showlegend=False)
        for trace in this_traces:
            fig.append_trace(trace, place_dic[method][0], place_dic[method][1])

    fig['layout'].update(height=1000, width=1000, legend=dict(orientation="h",
                                                             font=dict(size=16)))
    plotly.offline.iplot(fig, filename='yolarela')
        
methods_to_show = ['Pattern Search MDS', 'MDS SMACOF', 'Spectral Embedding', 'LLE', 'ISOMAP', 'Truncated SVD']
create_subplots3ds_emodb(X_RQA3, Y_RQA3, methods_to_show)
        
#     layout = dict(title = title)
#     fig = dict(data=traces, layout=layout)
#     plotly.offline.iplot(fig, filename=title)

In [ ]:
create_subplots3ds_emodb(X_Emobase3, Y_Emobase3, methods_to_show)


In [ ]:
create_subplots3ds_emodb(X_Fused3, Y_fused3, methods_to_show)

# Create IEMOCAP manifolds on 3d space

In [ ]:
def get_one_scatter_3d_traces(title, selected_speakers, emotions_xy_pairs1, 
                                                    emotions_xy_pairs2, showlegend=False):
    marker_size = 3
    colors = ['green', 'magenta', 'blue',   'orange', 'grey', 'magenta', 'black']
    traces = []
    sorted_emotions = sorted(emotions_xy_pairs1.keys())
    counter = 0
    for emotion in sorted_emotions:
        xypairs = emotions_xy_pairs1[emotion]
        trace = go.Scatter3d(
            x = xypairs['xs'],
            y = xypairs['ys'],
            z = xypairs['zs'],
            name = selected_speakers[0] + ' ' + emotion,
            legendgroup = emotion,
            mode = 'markers',
            opacity = 0.5,
            marker = dict(symbol = 'x', size = marker_size, color = colors[counter]),
            showlegend = showlegend
        )
        counter += 1
        traces.append(trace)
        
    counter = 0
    for emotion in sorted_emotions:
        xypairs = emotions_xy_pairs2[emotion]
        trace = go.Scatter3d(
            x = xypairs['xs'],
            y = xypairs['ys'],
            z = xypairs['zs'],
            name = selected_speakers[1] + ' ' + emotion,
            legendgroup = emotion,
            mode = 'markers',
            opacity = 0.5,
            marker = dict(symbol = 'circle', size = marker_size, color = colors[counter]),
            showlegend = showlegend
        )
        counter += 1
        traces.append(trace)    
    return traces

def create_subplots3ds_iemocap(Xs, Y1_3d, Y2_3d, methods_to_show, selected_speakers):
    
    if len(methods_to_show) == 6:
        pass 
    else: 
        raise IndexError('Not 6 plots to produce')
        
    fig = tools.make_subplots(rows=3, cols=2,
#                              subplot_titles=methods_to_show,
                             horizontal_spacing = 0.01, vertical_spacing = 0.1,
                             specs=[[{'is_3d': True}, {'is_3d': True}],
                                    [{'is_3d': True}, {'is_3d': True}],
                                    [{'is_3d': True}, {'is_3d': True}]])    

    legend_shower = methods_to_show[-1]
    
    row = 1
    place_dic = {}
    for k, method in enumerate(methods_to_show):
        if k % 2: 
            place_dic[method] = (row, 2)
            row += 1
        else:
            place_dic[method] = (row, 1)
            
    for method in methods_to_show:
        X_dic = Xs[method]
        emotions_xy_pairs1 = get_xy_pairs_based_on_class_3d(X_dic['X1'], Y1_3d)
        emotions_xy_pairs2 = get_xy_pairs_based_on_class_3d(X_dic['X2'], Y2_3d)
        
        if legend_shower == method:
            print legend_shower
            this_traces = get_one_scatter_3d_traces(method, selected_speakers, emotions_xy_pairs1, 
                                                    emotions_xy_pairs2, showlegend=True)
        else:
            this_traces = get_one_scatter_3d_traces(method, selected_speakers, emotions_xy_pairs1, 
                                                    emotions_xy_pairs2, showlegend=False)
            
        for trace in this_traces:
            fig.append_trace(trace, place_dic[method][0], place_dic[method][1])
        
    fig['layout'].update(height=1000, width=1000, legend=dict(orientation="h",
                                                             font=dict(size=20)),
                        title = 'From left to right: '+ ', '.join([x if not x == 'Spectral Embedding' else 'Spectral Clustering'
                                             for x in methods_to_show]))
    plotly.offline.iplot(fig, filename='yolarela')
        
methods_to_show = ['Pattern Search MDS', 'MDS SMACOF', 'Spectral Embedding', 'LLE', 'ISOMAP', 'Truncated SVD']
create_subplots3ds_iemocap(results_for_methods_3d, Y1_3d, Y2_3d, methods_to_show, selected_speakers)